# Import Packages

In [ ]:
import os

import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torchvision.utils import save_image

from architecture import MNIST_autoencoder as mnist
from functionalities import filemanager as fm
from functionalities import dataloader as dl
from functionalities import loss as lo
from functionalities import gpu 
from functionalities import plot as pl
from tqdm import tqdm_notebook as tqdm

# Pretraining Setup

In [ ]:

num_epochs = 100
batch_size = 128
learning_rate = 1e-3
milestones = [10 * x for x in range(1, 11)]
number_dev = 0

device = gpu.get_device(number_dev)


if not os.path.exists('./mlp_img'):
    os.mkdir('./mlp_img')


def to_img(x):
    #x = 0.5 * (x + 1)
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 1, 28, 28)
    return x

def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform(m.weight)
        m.bias.data.fill_(0.01)

In [ ]:
trainset, testset, classes = dl.load_mnist()
trainloader, validloader, testloader = dl.make_dataloaders(trainset, testset, batch_size)

# Training

In [ ]:

bottleneck_train_log = []
bottleneck_test_log = []    
    
for bottleneck in [1, 2, 3, 4, 6, 8, 12, 16, 24, 32, 48, 64]:
    print('bottleneck dimension: {}'.format(bottleneck))
    model = mnist.mnist_autoencoder_deep_1024(bottleneck).to(device)
    model.apply(init_weights)
    #criterion = nn.MSELoss() # l2 loss
    #criterion = lo.l1_loss() # l1 loss
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=milestones, gamma=0.1)
    
    train_loss_log = []
    test_loss_log = []


    for epoch in range(num_epochs):
        scheduler.step()
        for data in trainloader:
            img, _ = data
            img = img.view(img.size(0), -1)
            img = Variable(img).cuda()
            # ===================forward=====================
            output = model(img)
            loss = lo.l1_loss(output, img)
            # ===================backward====================
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        # ===================log========================
        train_loss_log.append(loss.data.item())
        print('epoch [{}/{}], train loss:{:.4f}'.format(epoch + 1, num_epochs, loss.data.item()))
        if epoch % 10 == 0 or epoch == (num_epochs - 1):
            pic = to_img(output.cpu().data)
            save_image(pic, './mlp_img/l1_image_deep_1024_{}_{}.png'.format(bottleneck, epoch))
            
        with torch.no_grad():
            test_loss = 0
            for data in testloader:
                img, _ = data
                img = img.view(img.size(0), -1)
                img = Variable(img).cuda()
                output = model(img)
                test_loss += lo.l1_loss(output, img).data.item()
                
            test_loss /= len(testloader)
            test_loss_log.append(test_loss)
            print('test loss:{:.4f}'.format(test_loss))
              
    bottleneck_train_log.append(train_loss_log[-1])
    bottleneck_test_log.append(test_loss_log[-1])

    fm.save_model(model, 'l1_sim_autoencoder_deep_1024_{}'.format(bottleneck))
    fm.save_weight(model, 'l1_sim_autoencoder_deep_1024_{}'.format(bottleneck))
    fm.save_variable([train_loss_log, test_loss_log], "l1_sim_autoencoder_deep_1024_{}".format(bottleneck))
    
fm.save_variable([bottleneck_train_log, bottleneck_test_log], "l1_sim_autoencoder_deep_1024_bottleneck")

# Plot Reconstruction and Difference Images Examples

In [ ]:
import matplotlib.pyplot as plt

num_img = 100
grid_row_size = 10

img, label = next(iter(testloader))
img = img.view(img.size(0), -1)
img = Variable(img).cuda()
modelname = 'com_classic_mnist'

for i in [1, 2, 3, 4, 6, 8, 12, 16, 24, 32, 48, 64]:
    print('bottleneck dimension: {}'.format(i))
    model = fm.load_model('l1_sim_autoencoder_{}'.format(i))
    output = model(img)

    
    original = to_img(img.cpu().data) 
    pic = to_img(output.cpu().data)

    print("Original Image:")
    pl.imshow(torchvision.utils.make_grid(original[:num_img].detach(), grid_row_size), filename="com_classic_mnist_{}_original".format(i))
    print("Reconstructed Image:")
    pl.imshow(torchvision.utils.make_grid(pic[:num_img].detach(), grid_row_size), filename="com_classic_mnist_{}_reconstructed".format(i))
    print("Difference:")
    diff_img = (original - pic + 1) / 2
    pl.imshow(torchvision.utils.make_grid(diff_img[:num_img].detach(), grid_row_size), filename="com_classic_mnist_{}_difference".format(i))
    

# Plot Recontruction Loss against Bottleneck Size

In [ ]:
train, test = fm.load_variable("l1_sim_autoencoder_bottleneck")
y = [train, test]
x = []
for loss in y:
    x.append([1, 2, 3, 4, 6, 8, 12, 16, 24, 32, 48, 64])

pl.plot([x for x in [1, 2, 3, 4, 6, 8, 12, 16, 24, 32, 48, 64]], y, 'latent dimension', 'loss', ['train', 'test'], 'Train & Test Reconstruction Loss History', 'loss_l1_bottleneck') 